In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle
import mlflow
import mlflow.keras
import keras_tuner as kt

In [3]:
pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

 Physical GPUs, 1 Logical GPUs


2022-06-29 18:59:15.381164: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-29 18:59:15.381271: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("graph-neural-hyperparameter-trial-6")

2022/06/29 19:40:12 INFO mlflow.tracking.fluent: Experiment with name 'graph-neural-hyperparameter-trial-6' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/10', experiment_id='10', lifecycle_stage='active', name='graph-neural-hyperparameter-trial-6', tags={}>

In [3]:
mlflow.tensorflow.autolog()

In [4]:
with open('data/train/x_train.pkl','rb') as f:
        x1=pickle.load(f)
with open('data/train/y_train.pkl','rb') as f:
        y1=pickle.load( f)
with open('data/valid/x_valid.pkl','rb') as f:
        x2=pickle.load(f)
with open('data/valid/y_valid.pkl','rb') as f:
        y2=pickle.load( f)
with open('data/test/x_test.pkl','rb') as f:
        x3=pickle.load(f)
with open('data/test/y_test.pkl','rb') as f:
        y3=pickle.load( f)

In [5]:
class GCN(tf.keras.layers.Layer):
    def __init__(self,inp_shape=45*60,out_shape=45*60,h=45, w=60,near=3):
        super(GCN, self).__init__()
        self.am=[[0 for i in range(w*h)] for j in range(h*w)]
        self.img=[[0 for i in range(w)] for j in range(h)]
  
        self.dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

        for i,v in enumerate(self.img):
           for j,v2 in enumerate(self.img[i]):
        # timg=[[0 for i in range(w)] for j in range(h)]
              for k in range(0,near+1):
                for d in self.dr:
                  posy=i+d[0]*k
                  posx=j+d[1]*k
                  if posy>=0 and posy<h and posx>=0 and posx<w:
            # timg[posy][posx]=1
                    self.am[i*w+j][posy*w+posx]=1
        # print(timg) 
        self.am=np.array(self.am) 
        self.D = np.diag(np.sum(self.am, axis=0)) 
        self.Dinv=np.linalg.inv(self.D)
        self.am=tf.Variable(self.am,trainable=True,dtype=tf.float32,name="ajm")
        self.Dinv=tf.Variable(self.Dinv,trainable=True,dtype=tf.float32,name="dm")
        # self.norm=tf.matmul(self.Dinv*self.am)
        self.w = self.add_weight(
            shape=(inp_shape, out_shape), initializer="random_normal", trainable=True,dtype=tf.float32,name='adjw'
        )
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        inputs=tf.cast(inputs,dtype=tf.float32)
        # inputs=tf.transpose()
        # print(inputs.shape)
        # print("inpshape",tf.transpose(inputs,perm=[1,0]).shape)
        # print(self.Dinv.shape)
        # print(self.am.shape)
        # print(self.w.shape)
        first_half=tf.matmul(inputs,tf.matmul(self.Dinv,self.am))
    
        return tf.matmul(first_half, self.w) 

In [6]:
def build_model(hp,near=3,lay=2,bs=2,n_out=7):
    inputs = tf.keras.Input( shape=(45,60,3))
    input=tf.transpose(inputs,perm=[3,0,1,2])
    r=input[0][:][:][:]
    g=input[1][:][:][:]
    b=input[2][:][:][:]
    r=tf.keras.layers.Flatten()(r)
    g=tf.keras.layers.Flatten()(g)
    b=tf.keras.layers.Flatten()(b)
    # print(inputs.shape)
    near=hp.Choice("number of nearest neighbour", [2, 3,4])
    grl=hp.Int( "no of layers for graph",min_value=lay, max_value=lay+15, step=4)
    for i in range(grl):
        r=GCN(near=near)(r)
        r=tf.keras.layers.Activation('relu',name="graph_layerr"+str(i+1))(r)
    r=tf.keras.layers.BatchNormalization()(r)
    for i in range(grl):
        g=GCN(near=near)(g)
        g=tf.keras.layers.Activation('relu',name="graph_layerg"+str(i+1))(g)
    g=tf.keras.layers.BatchNormalization()(g)
    
    for i in range(grl):
        b=GCN(near=near)(b)
        b=tf.keras.layers.Activation('relu',name="graph_layerb"+str(i+1))(b)
    b=tf.keras.layers.BatchNormalization()(b)
    x=tf.stack([r,g,b])
    x=tf.transpose(x,perm=[1,2,0])
    x=tf.keras.layers.Dense(units=1,activation="relu",name="denseshrink")(x)
    # print("at",x.shape)
    x=tf.squeeze(x,axis=-1)
   
    val=hp.Int( "no of layers for graph",min_value=lay, max_value=lay+15, step=4)
    for i in range(val):
        ut=hp.Int('units', min_value=32, max_value=128, step=32)
        al=hp.Choice("activation", ["relu", "tanh"])
        x=tf.keras.layers.Dense(units=ut,activation=al,name="dense"+str(i))(x)
        with mlflow.start_run(nested=True):
            mlflow.log_param("no of layers for dense", val)
            mlflow.log_param("no of units", ut)
            mlflow.log_param("type of activation in dense", al)
    output=tf.keras.layers.Dense(n_out,activation="softmax",name="out_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output, name="graph_neuralnetwork")
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    metrics=["accuracy"],
     )
    with mlflow.start_run(nested=True):
         mlflow.log_param("nearest", near)
         mlflow.log_param("no of layers for dense", val)
         mlflow.log_param("no of layers for graph", grl)
         mlflow.log_param("learning_rate", val)
    return model

In [28]:

hp = kt.HyperParameters()

In [19]:
with mlflow.start_run(nested=True):
    model=build_model(hp)

In [11]:
with mlflow.start_run(nested=True):
    tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=2,
    executions_per_trial=1,
    overwrite=True,
    directory="model/hyperparam",
    project_name="graph_neural_net",
)

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
number of nearest neighbour (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 4], 'ordered': True}
no of layers for graph (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 17, 'step': 4, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:

tuner.search(x1, y1, epochs=2, validation_data=(x2, y2))

Trial 2 Complete [00h 13m 25s]
val_accuracy: 0.16262754797935486

Best val_accuracy So Far: 0.16262754797935486
Total elapsed time: 00h 26m 18s
INFO:tensorflow:Oracle triggered exit


In [14]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

In [15]:
best_model.summary()

Model: "graph_neuralnetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 45, 60, 3)]  0           []                               
                                                                                                  
 tf.compat.v1.transpose (TFOpLa  (3, None, 45, 60)   0           ['input_1[0][0]']                
 mbda)                                                                                            
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 45, 60)      0           ['tf.compat.v1.transpose[0][0]'] 
 ingOpLambda)                                                                                     
                                                                                

In [16]:
best_model.save('/Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/hyperparam/gcn-hyp')

INFO:tensorflow:Assets written to: /Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/hyperparam/gcn-hyp/assets
